## 6. Decision tree

In [34]:
# Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeRegressor
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import xgboost as xgb

## 6.1 Data preparation

In [2]:
data = "https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv"

In [3]:
!wget $data

--2023-10-23 23:49:26--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 2606:50c0:8003::154, 2606:50c0:8001::154, 2606:50c0:8000::154, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|2606:50c0:8003::154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘housing.csv.3’

housing.csv.3       100%[===================>]   1.36M  --.-KB/s    in 0.09s   

2023-10-23 23:49:26 (14.5 MB/s) - ‘housing.csv.3’ saved [1423529/1423529]



In [4]:
# Load data
df = pd.read_csv(data)

In [5]:
# Filter the data based on `ocean_proximity`
df = df[df['ocean_proximity'].isin(['<1H OCEAN', 'INLAND'])]

# Prepare the data
# Fill missing values with zeros
df.fillna(0, inplace=True)

# Apply the log transform to `median_house_value`
df['median_house_value'] = np.log1p(df['median_house_value'])  # using log1p for numerical stability

# Do train/validation/test split with 60%/20%/20% distribution
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)  # 0.25 x 0.8 = 0.2

# Separate target variable
y_train = df_train['median_house_value'].values
y_val = df_val['median_house_value'].values
y_test = df_test['median_house_value'].values

# Remove target column from the data
del df_train['median_house_value']
del df_val['median_house_value']
del df_test['median_house_value']

# Use DictVectorizer to convert dataframes to matrices
dicts_train = df_train.to_dict(orient='records')
dicts_val = df_val.to_dict(orient='records')
dicts_test = df_test.to_dict(orient='records')

dv = DictVectorizer(sparse=True)
X_train = dv.fit_transform(dicts_train)
X_val = dv.transform(dicts_val)
X_test = dv.transform(dicts_test)


## Question 1


In [6]:
# Train the DecisionTreeRegressor
dt = DecisionTreeRegressor(max_depth=1)
dt.fit(X_train, y_train)

DecisionTreeRegressor(max_depth=1)

In [7]:
# Find out the feature used for splitting the data
split_feature_index = dt.tree_.feature[0]
split_feature_name = dv.feature_names_[split_feature_index]

In [8]:
print(f"Feature used for splitting: {split_feature_name}")

Feature used for splitting: ocean_proximity=<1H OCEAN


## Question 2

In [9]:
# 1. Train the Random Forest model on the training data
rf = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestRegressor(n_estimators=10, n_jobs=-1, random_state=1)

In [10]:
# 2. Predict values on the validation data
y_pred = rf.predict(X_val)

In [11]:
# 3. Calculate the RMSE using these predicted values
rmse = np.sqrt(mean_squared_error(y_val, y_pred))

In [12]:
print(f"RMSE: {round(rmse,4)}")


RMSE: 0.2452


## Question 3

In [13]:
best_rmse = float('inf')
best_n = None

In [15]:
for n in range(10, 201, 10): 
    rf = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rf.fit(X_train, y_train)

    y_pred = rf.predict(X_val)
    rmse = np.sqrt(mean_squared_error(y_val, y_pred))

    if rmse < best_rmse - 0.001:
        best_rmse = rmse
        best_n = n
    else:
        break
print(f"RMSE stops at the following n_estimators: {best_n} with the following RMSE: {round(best_rmse,3)}")

RMSE stops at the following n_estimators: 40 with the following RMSE: 0.235


## Question 4

In [20]:
best_rmse = float('inf')
best_n = None
best_max_depth = None
max_depth = [10, 15, 20, 25]

In [22]:
for d in max_depth: 
    rmses = []
    for n in range(10, 201, 10): 
        rf = RandomForestRegressor(n_estimators=n, random_state=1,  max_depth=d, n_jobs=-1)
        rf.fit(X_train, y_train)
    
        y_pred = rf.predict(X_val)
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))

        rmses.append(rmse)
        
    mean_rmse = np.mean(rmses)

    if mean_rmse < best_rmse:
        best_rmse = mean_rmse
        best_max_depth = d
        best_estimator = n 
        
print(f"RMSE stops at the following n_estimators: {best_estimator} with the following mean RMSE: {round(best_rmse,3)} and best max depth {best_max_depth}")

RMSE stops at the following n_estimators: 200 with the following mean RMSE: 0.235 and best max depth 25


## Question 5

In [24]:
# 1. Train the RandomForestRegressor with the specified parameters
rf = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
rf.fit(X_train, y_train)

# 2. Retrieve the feature_importances_ attribute from the trained model
importances = rf.feature_importances_

# 3. Match the importance scores with the feature names
features = dv.feature_names_
feature_importance = list(zip(features, importances))

# Sort by importance
sorted_feature_importance = sorted(feature_importance, key=lambda x: x[1], reverse=True)

# Print out the most important feature
print(f"Most important feature: {sorted_feature_importance[0][0]}")

Most important feature: median_income


## Question 6

In [36]:
# 1. Create DMatrix for train and validation
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

In [37]:
# 2. Create a watchlist
watchlist = [(dtrain, 'train'), (dval, 'val')]


In [44]:
# 3. Train model with eta=0.3
model_03 = xgb.train(
    xgb_params_03, 
    dtrain, 
    num_boost_round=100, 
    evals=watchlist, 
    early_stopping_rounds=10,  # Stop if no improvement in RMSE for 10 consecutive rounds
    verbose_eval=5
)
rmse_03 = model_03.best_score

[0]	train-rmse:0.44350	val-rmse:0.44250
[5]	train-rmse:0.25338	val-rmse:0.27463
[10]	train-rmse:0.21444	val-rmse:0.25179
[15]	train-rmse:0.19858	val-rmse:0.24522
[20]	train-rmse:0.18524	val-rmse:0.23978
[25]	train-rmse:0.17757	val-rmse:0.23830
[30]	train-rmse:0.16888	val-rmse:0.23570
[35]	train-rmse:0.16113	val-rmse:0.23416
[40]	train-rmse:0.15542	val-rmse:0.23318
[45]	train-rmse:0.14941	val-rmse:0.23190
[50]	train-rmse:0.14536	val-rmse:0.23225
[55]	train-rmse:0.14150	val-rmse:0.23197


In [45]:
# 4. Train model with eta=0.1
model_01 = xgb.train(
    xgb_params_01, 
    dtrain, 
    num_boost_round=100, 
    evals=watchlist, 
    early_stopping_rounds=10,  # Stop if no improvement in RMSE for 10 consecutive rounds
    verbose_eval=5
)
rmse_01 = model_01.best_score

[0]	train-rmse:0.52449	val-rmse:0.52045
[5]	train-rmse:0.37822	val-rmse:0.38151
[10]	train-rmse:0.30326	val-rmse:0.31427
[15]	train-rmse:0.26538	val-rmse:0.28380
[20]	train-rmse:0.24512	val-rmse:0.26882
[25]	train-rmse:0.23026	val-rmse:0.25997
[30]	train-rmse:0.21887	val-rmse:0.25266
[35]	train-rmse:0.21020	val-rmse:0.24826
[40]	train-rmse:0.20392	val-rmse:0.24539
[45]	train-rmse:0.19814	val-rmse:0.24293
[50]	train-rmse:0.19215	val-rmse:0.24020
[55]	train-rmse:0.18809	val-rmse:0.23878
[60]	train-rmse:0.18457	val-rmse:0.23791
[65]	train-rmse:0.18063	val-rmse:0.23698
[70]	train-rmse:0.17741	val-rmse:0.23622
[75]	train-rmse:0.17468	val-rmse:0.23510
[80]	train-rmse:0.17242	val-rmse:0.23453
[85]	train-rmse:0.17014	val-rmse:0.23404
[90]	train-rmse:0.16797	val-rmse:0.23332
[95]	train-rmse:0.16562	val-rmse:0.23276
[99]	train-rmse:0.16323	val-rmse:0.23209


In [46]:
# 6. Compare the two RMSE scores
if rmse_03 < rmse_01:
    best_eta = "0.3"
elif rmse_03 > rmse_01:
    best_eta = "0.1"
else:
    best_eta = "Both give equal value"

print(f"Best eta: {best_eta}")



Best eta: 0.3
